<a href="https://colab.research.google.com/github/869277160/LangChainPlayGround/blob/main/2_%E6%A8%A1%E5%9E%8B%E3%80%81%E6%8F%90%E7%A4%BA%E5%92%8C%E8%A7%A3%E6%9E%90%E5%99%A8_Models%2C_Prompts_and_Output_Parsers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第二章 模型，提示和输出解释器

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、直接使用OpenAI](#二、直接使用OpenAI)
     - [2.1 计算1+1](#2.1-计算1+1)
     - [2.2 用美式英语表达海盗邮件](#2.2-用美式英语表达海盗邮件)
     - [2.3 中文版](#2.3-中文版)
 - [三、通过LangChain使用OpenAI](#三、通过LangChain使用OpenAI)
     - [3.1 模型](#3.1-模型)
     - [3.2 提示模板](#3.2-提示模板)
         - [3.2.1 使用LangChain提示模版](#3.2.1-使用LangChain提示模版)
         - [3.2.2 中文版](#3.2.2-中文版)
         - [3.2.2 为什么需要提示模版](#3.2.2-为什么需要提示模版)
     - [3.3 输出解析器](#3.3-输出解析器)
         - [3.3.1 如果没有输出解析器](#3.3.1-如果没有输出解析器)
         - [3.3.2 中文版](#3.3.2-中文版)
         - [3.3.3 LangChain输出解析器](#3.3.3-LangChain输出解析器)
         - [3.3.4 中文版](#3.3.4-中文版)
 - [四、补充材料](#四、补充材料)
     - [4.1 链式思考推理(ReAct)](#4.1-链式思考推理(ReAct))


## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key"
    </p>
  
  替换"your_api_key"为你自己的 API Key


这里将chatgpt 转换为chatglm

In [1]:
# 下载需要的包python-dotenv和openai
# 如果你需要查看安装过程日志，可删除 -q
# !pip install -q python-dotenv
!pip install -q openai

In [2]:
import os
import openai
# from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
# _ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
# openai.api_key = os.environ['OPENAI_API_KEY']
# openai.api_base = "https://api.openai.com/v1" # 换成代理，一定要加v1
openai.api_base = "https://openkey.cloud/v1" # 换成代理，一定要加v1
# openai.api_key = "API_KEY"
openai.api_key = "sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5"


In [3]:
# from LangChainChatGLM import *
# from LangChainChatGLM import ChatGLM

# llm = ChatGLM()

# testing code for the model
# for i in range(10):
#     human_input = input("Human: ")

#     begin_time = time.time() * 1000
#     # 请求模型
#     response = llm(human_input, stop=["you"])
#     end_time = time.time() * 1000
#     used_time = round(end_time - begin_time, 3)
#     logging.info(f"chatGLM process time: {used_time}ms")

#     print(f"ChatGLM: {response}")

## 二、直接使用OpenAI

我们先从直接调用OpenAI的API开始。

`get_completion`函数是基于`openai`的封装函数，对于给定提示（prompt）输出相应的回答。其包含两个参数
   
   - `prompt` 必需输入参数。 你给模型的**提示，可以是一个问题，可以是你需要模型帮助你做的事**（改变文本写作风格，翻译，回复消息等等）。
   - `model` 非必需输入参数。默认使用gpt-3.5-turbo。你也可以选择其他模型。
   
这里的提示对应我们给chatgpt的问题，函数给出的输出则对应chatpgt给我们的答案。

In [4]:
# def get_completion_from_messages(messages,
#                                  model="gpt-3.5-turbo",
#                                 temperature=0,
#                                 max_tokens=500):
#     '''
#     封装一个访问 OpenAI GPT3.5 的函数

#     参数:
#     messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
#     model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
#     temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
#     max_tokens: 这决定模型输出的最大的 token 数。
#     '''
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=temperature, # 这决定模型输出的随机程度
#         max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
#     )
#     return response.choices[0].message["content"]

def get_completion(prompt, model="gpt-3.5-turbo"):

    messages = [{"role": "user", "content": prompt}]

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

### 2.1 计算1+1

我们来一个简单的例子 - 分别用中英文问问模型

- 中文提示(Prompt in Chinese)： `1+1是什么？`
- 英文提示(Prompt in English)： `What is 1+1?`

In [7]:
# 中文
get_completion("1+1是什么？")

'1+1等于2。'

In [8]:
# 英文
get_completion("What is 1+1?")

'1+1 equals 2.'

### 2.2 用美式英语表达海盗邮件

上面的简单例子，模型`gpt-3.5-turbo`对我们的关于1+1是什么的提问给出了回答。

现在我们来看一个复杂一点的例子：

假设我们是电商公司员工，我们的顾客是一名海盗A，他在我们的网站上买了一个榨汁机用来做奶昔，在制作奶昔的过程中，奶昔的盖子飞了出去，弄得厨房墙上到处都是。于是海盗A给我们的客服中心写来以下邮件：`customer_email`

In [9]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# 中文非正式用语
customer_email = """
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

我们的客服人员对于海盗的措辞表达觉得有点难以理解。 现在我们想要实现两个小目标：

- 让模型用美式英语的表达方式将海盗的邮件进行翻译，客服人员可以更好理解。*这里海盗的英文表达可以理解为英文的方言，其与美式英语的关系，就如四川话与普通话的关系。
- 让模型在翻译是用平和尊重的语气进行表达，客服人员的心情也会更好。

根据这两个小目标，定义一下文本表达风格：`style`

In [10]:
# 美式英语 + 平静、尊敬的语调
style = """American English \
in a calm and respectful tone
"""

下一步需要做的是将`customer_email`和`style`结合起来构造我们的提示:`prompt`

In [11]:
# 要求模型根据给出的语调进行转化
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！
```



`prompt` 构造好了，我们可以调用`get_completion`得到我们想要的结果 - 用平和尊重的语气，美式英语表达的海盗语言邮件

In [12]:
response = get_completion(prompt)
response

"Oh, I'm really frustrated because the lid of my blender fell off and splattered the milkshake all over the kitchen wall! To make matters worse, the warranty doesn't cover the cost of cleaning the kitchen. I could really use your help right now, buddy!"

对比语言风格转换前后，用词更为正式，替换了极端情绪的表达，并表达了感谢。

✨ 你可以尝试修改提示，看可以得到什么不一样的结果😉

### 2.3 中文版

In [13]:
# 普通话 + 平静、尊敬的语调
style = """正式普通话 \
用一个平静、尊敬的语调
"""

In [14]:
# 要求模型根据给出的语调进行转化
prompt = f"""把由三个反引号分隔的文本\
翻译成一种{style}风格。
文本: ```{customer_email}```
"""

print(prompt)



把由三个反引号分隔的文本翻译成一种正式普通话 用一个平静、尊敬的语调
风格。
文本: ```
阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！
```



In [15]:
response = get_completion(prompt)
response

'尊敬的朋友，我感到不安，因为我的搅拌机盖子不慎掉落，导致奶昔溅到了厨房的墙壁上！更加令人糟心的是，保修并不包含厨房清洁的费用。此刻，我需要你的帮助，亲爱的伙伴！'

## 三、通过LangChain使用OpenAI

在前面一部分，我们通过封装函数`get_completion`直接调用了OpenAI完成了对方言邮件进行了的翻译，得到用平和尊重的语气、正式的普通话表达的邮件。

让我们尝试使用LangChain来实现相同的功能。

In [16]:
# 如果你需要查看安装过程日志，可删除 -q
# --upgrade 让我们可以安装到最新版本的 langchain
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00


### 3.1 模型

从`langchain.chat_models`导入`OpenAI`的对话模型`ChatOpenAI`。 除去OpenAI以外，`langchain.chat_models`还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://python.langchain.com/en/latest/modules/models/chat/integrations.html)。

这里依旧使用chatglm


In [17]:
from langchain.chat_models import ChatOpenAI

In [21]:
# openai.api_base = "https://openkey.cloud/v1" # 换成代理，一定要加v1
# # openai.api_key = "API_KEY"
# openai.api_key = "sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5"

# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
chat = ChatOpenAI(
    openai_api_base = "https://openkey.cloud/v1",
    openai_api_key = "sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5",
    temperature=0.0,
    )
chat
chat.model_name

'gpt-3.5-turbo'

上面的输出显示ChatOpenAI的默认模型为`gpt-3.5-turbo`

### 3.2 提示模板

在前面的例子中，我们通过[f字符串](https://docs.python.org/zh-cn/3/tutorial/inputoutput.html#tut-f-strings)把Python表达式的值`style`和`customer_email`添加到`prompt`字符串内。

```python
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""
```
`langchain`提供了接口方便快速的构造和使用提示。现在我们来看看如何使用`langchain`来构造提示。

#### 3.2.1 使用LangChain提示模版

##### 1️⃣ 构造提示模版字符串
我们构造一个提示模版字符串：`template_string`

In [22]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

##### 2️⃣ 构造LangChain提示模版
我们调用`ChatPromptTemplatee.from_template()`函数将上面的提示模版字符`template_string`转换为提示模版`prompt_template`

In [23]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain import PromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [24]:
print(prompt_template.messages[0].prompt)

input_variables=['style', 'text'] output_parser=None partial_variables={} template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n' template_format='f-string' validate_template=True


从上面的输出可以看出，`prompt_template` 有两个输入变量： `style` 和 `text`。

In [25]:
print(prompt_template.messages[0].prompt.input_variables)

['style', 'text']


##### 3️⃣ 使用模版得到客户消息提示

langchain提示模版`prompt_template`需要两个输入变量： `style` 和 `text`。 这里分别对应
- `customer_style`: 我们想要的顾客邮件风格
- `customer_email`: 顾客的原始邮件文本。

In [26]:
customer_style = """American English \
in a calm and respectful tone
"""

In [27]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

对于给定的`customer_style`和`customer_email`, 我们可以使用提示模版`prompt_template`的`format_messages`方法生成想要的客户消息`customer_messages`。

In [28]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [29]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


可以看出`customer_messages`变量类型为列表(`list`)，而列表里的元素变量类型为langchain自定义消息(`langchain.schema.HumanMessage`)。

打印第一个元素可以得到如下:

In [30]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [ ]:
# 中文提示
from langchain.prompts import ChatPromptTemplate

template_string = """把由三个反引号分隔的文本\
翻译成一种{style}风格。\
文本: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""

customer_email = """
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


print(customer_messages[0])

##### 4️⃣ 调用chat模型转换客户消息风格

现在我们可以调用[模型](#model)部分定义的chat模型来实现转换客户消息风格。到目前为止，我们已经实现了在前一部分的任务。

In [31]:
customer_response = chat(customer_messages)

In [32]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


In [33]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


##### 5️⃣ 使用模版得到回复消息提示

接下来，我们更进一步，将客服人员回复的消息，转换为海盗的语言风格，并确保消息比较有礼貌。

这里，我们可以继续使用第2️⃣步构造的langchain提示模版，来获得我们回复消息提示。

In [34]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [35]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [36]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



##### 6️⃣ 调用chat模型转换回复消息风格

调用[模型](#model)部分定义的chat模型来转换回复消息风格

In [37]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, matey! I regret to inform ye that the warranty be not coverin' the costs o' cleanin' yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to secure the lid afore startin' it. Aye, tough luck, me heartie! Fare thee well!


#### 3.2.2 中文版

In [ ]:
service_reply = """嘿，顾客， \
保修不包括厨房的清洁费用， \
因为您在启动搅拌机之前 \
忘记盖上盖子而误用搅拌机, \
这是您的错。 \
倒霉！ 再见！
"""

service_style_pirate = """\
一个有礼貌的语气 \
使用正式的普通话 \
"""
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

#### 3.2.2 为什么需要提示模版

在应用于比较复杂的场景时，提示可能会非常长并且包含涉及许多细节。**使用提示模版，可以让我们更为方便地重复使用设计好的提示**。

下面给出了一个比较长的提示模版案例。学生们线上学习并提交作业，通过以下的提示来实现对学生的提交的作业的评分。

In [ ]:
# 英文版
prompt = """ Your task is to determine if the student's solution is correct or not

    To solve the problem do the following:
    - First, workout your own solution to the problem
    - Then compare your solution to the student's solution
    and evaluate if the sudtent's solution is correct or not.
    ...
    Use the following format:
    Question:
    ```
    question here
    ```
    Student's solution:
    ```
    student's solution here
    ```
    Actual solution:
    ```
    ...
    steps to work out the solution and your solution here
    ```
    Is the student's solution the same as acutal solution \
    just calculated:
    ```
    yes or no
    ```
    Student grade
    ```
    correct or incorrect
    ```

    Question:
    ```
    {question}
    ```
    Student's solution:
    ```
    {student's solution}
    ```
    Actual solution:

    """

In [ ]:
# 中文版
prompt = """ 你的任务是判断学生的解决方案是正确的还是不正确的

要解决该问题，请执行以下操作：
 - 首先，制定自己的问题解决方案
 - 然后将您的解决方案与学生的解决方案进行比较
 并评估学生的解决方案是否正确。
...
使用下面的格式:

问题:
```
问题文本
```
学生的解决方案:
```
学生的解决方案文本
```
实际解决方案:
```
...
制定解决方案的步骤以及您的解决方案请参见此处
```
学生的解决方案和实际解决方案是否相同 \
只计算：
```
是或者不是
```
学生的成绩
```
正确或者不正确
```

问题:
```
{question}
```
学生的解决方案:
```
{student's solution}
```
实际解决方案:

"""

此外，LangChain还提供了提示模版用于一些常用场景。比如自动摘要、问答、连接到SQL数据库、连接到不同的API. 通过使用LongChain内置的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示。

最后，我们在建立大模型应用时，通常希望模型的输出为给定的格式，比如在输出使用特定的关键词来让输出结构化。 下面为一个使用大模型进行链式思考推理例子，对于问题：*What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?*

通过使用LangChain库函数，输出采用"Thought"（思考）、"Action"（行动）、"Observation"（观察）作为链式思考推理的关键词，让输出结构化。

```
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions

Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]

Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]
```

在补充材料中，可以查看使用LangChain和OpenAI进行链式思考推理的另一个代码实例。

### 3.3 输出解析器

#### 3.3.1 如果没有输出解析器

对于给定的评价`customer_review`, 我们希望提取信息，并按以下格式输出：

```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

##### 1️⃣ 构造提示模版字符串

In [ ]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

##### 2️⃣ 构造langchain提示模版

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

##### 3️⃣ 使用模版得到提示消息

In [ ]:
messages = prompt_template.format_messages(text=customer_review)

##### 4️⃣ 调用chat模型提取信息

In [ ]:
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

##### 📝 分析与总结
`response.content`类型为字符串（`str`），而并非字典(`dict`), 直接使用`get`方法会报错。因此，我们需要输出解释器。

In [ ]:
type(response.content)

In [ ]:
response.content.get('gift')

#### 3.3.2 中文版

In [ ]:
from langchain.prompts import ChatPromptTemplate

customer_review = """\
这款吹叶机非常神奇。 它有四个设置：\
吹蜡烛、微风、风城、龙卷风。 \
两天后就到了，正好赶上我妻子的\
周年纪念礼物。 \
我想我的妻子会喜欢它到说不出话来。 \
到目前为止，我是唯一一个使用它的人，而且我一直\
每隔一天早上用它来清理草坪上的叶子。 \
它比其他吹叶机稍微贵一点，\
但我认为它的额外功能是值得的。
"""

review_template = """\
对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？ \
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天\
到达？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，\
并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

文本: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)


In [ ]:
messages = prompt_template.format_messages(text=customer_review)

chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

#### 3.3.3 LangChain输出解析器

##### 1️⃣ 构造提示模版字符串

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

##### 2️⃣ 构造langchain提示模版

In [ ]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

##### 🔥 构造输出解析器

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")


response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

##### 3️⃣ 使用模版得到提示消息

In [ ]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

##### 4️⃣ 调用chat模型提取信息

In [ ]:
response = chat(messages)
print(response.content)

##### 5️⃣ 使用输出解析器解析输出

In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

##### 📝 分析与总结
`output_dict`类型为字典(`dict`), 可直接使用`get`方法。这样的输出更方便下游任务的处理。

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')

#### 3.3.4 中文版

In [ ]:
# 中文
review_template_2 = """\
对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: {text}

{format_instructions}
"""

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="礼物",
                             description="这件物品是作为礼物送给别人的吗？\
                            如果是，则回答 是的，\
                            如果否或未知，则回答 不是。")

delivery_days_schema = ResponseSchema(name="交货天数",
                                      description="产品需要多少天才能到达？\
                                      如果没有找到该信息，则输出-1。")

price_value_schema = ResponseSchema(name="价钱",
                                    description="提取有关价值或价格的任何句子，\
                                    并将它们输出为逗号分隔的 Python 列表")


response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)
print(messages[0].content)

In [ ]:
response = chat(messages)
print(response.content)


In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

## 四、补充材料

### 4.1 链式思考推理(ReAct)
参考资料：[ReAct (Reason+Act) prompting in OpenAI GPT and LangChain](https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/)

In [38]:
!pip install -q wikipedia

  Preparing metadata (setup.py) ... done


In [40]:
from langchain.docstore.wikipedia import Wikipedia
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# 使用大语言模型
llm = OpenAI(
  openai_api_base = "https://openkey.cloud/v1",
  openai_api_key = "sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5",
  model_name="gpt-3.5-turbo",
  temperature=0,
)
# chat = ChatOpenAI(
#     openai_api_base = "https://openkey.cloud/v1",
#     openai_api_key = "sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5",
#     temperature=0.0,
#     )
# chat

# 初始化ReAct代理
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
)


question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent_executor.run(question)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: I need to search David Chanoff and find the U.S. Navy admiral he collaborated with. Then I need to find the president under whom the admiral served as the ambassador to the United Kingdom.
Action: Search[David Chanoff]
Observation: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought:David Chanoff has collaborated with several individuals, including Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good, and Felix Zandman. I need to

'Bill Clinton'